In [3]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [5]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry import union
from arcgis.geometry.filters import intersects
from arcgis.mapping import generate_classbreaks
from datetime import datetime, timedelta, timezone
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.fires import aggregate, query
from georapid.formats import OutFormat

## Creating a client

**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!


In [6]:
host = 'geofires.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Query the aggregated features of yesterday

In [7]:
date_of_interest = datetime.now(timezone.utc) - timedelta(days=1)
aggregated_fires_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
aggregated_fires_fset = FeatureSet.from_dict(aggregated_fires_dict)
aggregated_fires_sdf = aggregated_fires_fset.sdf
aggregated_fires_sdf

,index,count,OBJECTID,SHAPE
0,0,2102,1,"{""rings"": [[[-13325811.463470597, 4037508.3427..."
1,1,1,2,"{""rings"": [[[1829633.102757085, 5787508.342799..."
2,2,2,3,"{""rings"": [[[-8346165.391710073, 412508.342799..."
3,7,25,4,"{""rings"": [[[97582.29518820718, 6287508.342799..."
4,10,1,5,"{""rings"": [[[-10294722.55022506, 1787508.34279..."
...,...,...,...,...
153,3077,1,154,"{""rings"": [[[-11593760.655901719, 3787508.3427..."
154,3166,1,155,"{""rings"": [[[-9645203.497386731, 5162508.34279..."
155,3195,1,156,"{""rings"": [[[-10944241.60306339, 4662508.34279..."
156,3202,1,157,"{""rings"": [[[-10944241.60306339, 4412508.34279..."


## Query the mentioned locations

In [8]:
mentioned_fires_dict = query(client, date_of_interest, OutFormat.ESRI)
mentioned_fires_fset = FeatureSet.from_dict(mentioned_fires_dict)
mentioned_fires_sdf = mentioned_fires_fset.sdf
mentioned_fires_sdf

,name,count,OBJECTID,SHAPE
0,"Los Angeles, California, United States",2238,1,"{""x"": -118.244, ""y"": 34.0522, ""spatialReferenc..."
1,"Kaposvar, Somogy, Hungary",1,2,"{""x"": 17.8, ""y"": 46.3667, ""spatialReference"": ..."
2,"Bogota, Cundinamarca, Colombia",3,3,"{""x"": -74.0833, ""y"": 4.6, ""spatialReference"": ..."
3,"City Of Nacogdoches, Texas, United States",2,4,"{""x"": -94.6505, ""y"": 31.6151, ""spatialReferenc..."
4,"Nacogdoches, Texas, United States",2,5,"{""x"": -94.6555, ""y"": 31.6035, ""spatialReferenc..."
...,...,...,...,...
1110,"Dorila, La Pampa, Argentina",1,1111,"{""x"": -63.7156, ""y"": -35.7757, ""spatialReferen..."
1111,"Silver Lake, New York, United States",1,1112,"{""x"": -75.3324, ""y"": 42.6015, ""spatialReferenc..."
1112,"Monaghan, Monaghan, Ireland",1,1113,"{""x"": -6.96667, ""y"": 54.25, ""spatialReference""..."
1113,"Metro Manila, Philippines (General), Philippines",1,1114,"{""x"": 121.0, ""y"": 14.5833, ""spatialReference"":..."


## Map news related to wildfires with thermal activities
We access the MODIS layer being hosted using ArcGIS Living Atlas. One specific layer manages the thermal events of the last 48 hours.

In [9]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)

In [10]:
gis = GIS()

In [12]:
item_id = 'b8f4033069f141729ffb298b7418b653'
thermal_item = gis.content.get(item_id)
thermal_item

<Item title:"Satellite (MODIS) Thermal Hotspots and Fire Activity" type:Feature Layer Collection owner:esri_livefeeds2>

## Query the thermal events using the union of aggregated news grid cells and also the date of interest
We are using the union of the aggregated news grid cells and the date of interest with a kind of interval +/-1 day. We expect a number of news being greater than 20 and a confidence above 70 for the thermal events.

In [13]:
aggregated_fires_union = union(aggregated_fires_sdf[aggregated_fires_sdf['count'] > 20].SHAPE.tolist())
spatial_filter = intersects(aggregated_fires_union[0])

In [14]:
thermal_layer = thermal_item.layers[0]
start_date_str = (date_of_interest - timedelta(days=1)).strftime('%Y-%m-%d')
end_date_str = (date_of_interest + timedelta(days=1)).strftime('%Y-%m-%d')
where_clause = f"CONFIDENCE > 70 AND ACQ_DATE BETWEEN DATE '{start_date_str}' AND DATE '{end_date_str}'"
thermal_events_sdf = thermal_layer.query(where=where_clause, geometry_filter=spatial_filter, as_df=True)
thermal_events_sdf

,OBJECTID,BRIGHTNESS,SCAN,TRACK,SATELLITE,CONFIDENCE,VERSION,BRIGHT_T31,FRP,ACQ_DATE,DAYNIGHT,HOURS_OLD,DAY_OF_ACQ,SHAPE
0,66111,344.86,3.62,1.78,T,100,6.1NRT,290.02,344.77,2025-01-10 04:21:00,N,57,2,"{""x"": -13191866.162700001, ""y"": 4042908.345299..."
1,66113,354.42,3.64,1.79,T,100,6.1NRT,294.03,478.65,2025-01-10 04:21:00,N,57,2,"{""x"": -13195820.230999999, ""y"": 4041785.8103, ..."
2,66117,334.93,3.64,1.79,T,100,6.1NRT,288.32,242.82,2025-01-10 04:21:00,N,57,2,"{""x"": -13196314.489500001, ""y"": 4043852.167000..."
3,66118,320.85,3.39,1.74,T,100,6.1NRT,282.47,125.91,2025-01-10 04:21:00,N,57,2,"{""x"": -13147603.306699999, ""y"": 4059653.323899..."
4,66119,341.0,3.62,1.78,T,100,6.1NRT,289.4,302.25,2025-01-10 04:21:00,N,57,2,"{""x"": -13192790.114500001, ""y"": 4044316.039700..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,90224,393.04,1.01,1.01,A,100,6.1NRT,321.71,210.44,2025-01-11 22:01:00,D,15,1,"{""x"": -13196576.0904, ""y"": 4044273.0129999965,..."
83,90226,467.05,1.01,1.01,A,100,6.1NRT,400.07,971.06,2025-01-11 22:01:00,D,15,1,"{""x"": -13197772.774799999, ""y"": 4044071.327699..."
84,90228,326.66,1.01,1.01,A,84,6.1NRT,298.28,20.73,2025-01-11 22:01:00,D,15,1,"{""x"": -13198958.327399999, ""y"": 4043872.335000..."
85,90231,357.66,1.01,1.01,A,99,6.1NRT,304.44,75.45,2025-01-11 22:01:00,D,15,1,"{""x"": -13196776.4654, ""y"": 4045457.650600001, ..."


In [15]:
joined_fires_sdf = aggregated_fires_sdf[['OBJECTID', 'count', 'SHAPE']].spatial.join(thermal_events_sdf[['CONFIDENCE', 'SHAPE']])
joined_fires_sdf

,OBJECTID,count,SHAPE,index_right,CONFIDENCE
0,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",0,100
1,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",1,100
2,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",2,100
3,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",3,100
4,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",4,100
...,...,...,...,...,...
82,11,64,"{""rings"": [[[-13109305.112524487, 3912508.3427...",24,100
83,11,64,"{""rings"": [[[-13109305.112524487, 3912508.3427...",25,80
84,11,64,"{""rings"": [[[-13109305.112524487, 3912508.3427...",26,100
85,11,64,"{""rings"": [[[-13109305.112524487, 3912508.3427...",27,72


In [32]:
reduced_fires_sdf = joined_fires_sdf.groupby(by=['OBJECTID']).agg({
    'count': 'first',
    'SHAPE': 'first',
    'CONFIDENCE': 'mean'
}).reset_index()
reduced_fires_sdf['events'] = joined_fires_sdf.groupby(by=['OBJECTID']).size().astype('Int32').values
reduced_fires_sdf

,OBJECTID,count,SHAPE,CONFIDENCE,events
0,1,2102,"{""rings"": [[[-13325811.463470597, 4037508.3427...",96.901235,81
1,11,64,"{""rings"": [[[-13109305.112524487, 3912508.3427...",85.166667,6


In [34]:
map_view = gis.map('California, USA')
map_view.basemap = 'osm'

plot_aggregated(map_view, reduced_fires_sdf)
thermal_events_sdf.spatial.plot(map_view)
map_view

MapView(layout=Layout(height='400px', width='100%'))